In [1]:
import folium
import ast
import numpy as np
import http.server
import socketserver
import webbrowser
import threading
import os
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
from pathlib import Path
from shapely.wkt import loads
from shapely.geometry import Point, Polygon
from shapely.geometry import mapping
from fastkml import kml

In [ ]:
#use this block if data has only GEO_IDs as reference from census data, otherwise skip to next block, joins census .csv to county polygon .csv for geo-references
def join_csv_files(file1, file2, key1='GEO_ID' , key2='GEOIDFQ', output_file= 'joined.csv'):
    df1 = pd.read_csv(file1)
    df2 = pd.read_csv(file2)
    merged_df = df1.merge(df2, left_on=key1, right_on=key2, how='inner')
    # Save to a new CSV file
    merged_df.to_csv(output_file, index=False)
    
#the below join should be replaced with the filepath to your tl_2024_us_county data file download, see github wiki
join_csv_files(file1, r"C:\Users\zachh\Downloads\tl_2024_us_county\tl_2024_us_county.csv", key1='GEO_ID', key2='GEOIDFQ', output_file= joined.csv)
print(f"Join complete! Merged file saved as {output_file}")

In [ ]:
# input your file path for input_file below, paste the same path in for output_file and change your file extension to .kml

input_file = r"C:\Users\zachh\Downloads\ACSST1Y2023.S2201_2025-02-16T201609\joined.csv"
output_file = r"C:\Users\zachh\Downloads\ACSST1Y2023.S2201_2025-02-16T201609\9joined.kml"

def convert_to_kml(input_file, output_file):
    gdf = None
    
    # Determine file format and preprocess data according to filetype
    if input_file.endswith('.csv'):
        df = pd.read_csv(input_file)
        if 'geometry' not in df.columns:
            raise ValueError("CSV file must contain a 'geometry' column with WKT format.")
        df['geometry'] = df['geometry'].apply(loads)
        gdf = gpd.GeoDataFrame(df, geometry='geometry')
    elif input_file.endswith(('.shp', '.geojson')):
        gdf = gpd.read_file(input_file)
    else:
        raise ValueError("Unsupported file format. Please provide a .csv, .shp, or .geojson file.")
    
    # Ensure gdf is a GeoDataFrame
    if not isinstance(gdf, gpd.GeoDataFrame):
        raise TypeError("Failed to create a valid GeoDataFrame from the input file.")
    
    k = kml.KML()
    doc = kml.Document()
    k.append(doc)
    
    for _, row in gdf.iterrows():
        if row.geometry is None:
            continue
        placemark = kml.Placemark()
        placemark.geometry = mapping(row.geometry)  # Handles points, lines, and polygons
        placemark.name = str(row.get('name', 'Unnamed'))  # Use 'name' column if available
        doc.append(placemark)
    # Write to KML file, oath can be specified by replacing output_file
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write(k.to_string(prettyprint=True))

convert_to_kml(input_file, output_file)
    
print(f"Conversion complete! KML saved as {output_file}")

In [2]:
# Define a localhost server port to avoid browser and python security issues
PORT = 8000

# defining the parent folder of the jupyter notebook as directory, must load your KML there
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
os.chdir(BASE_DIR)
Handler = http.server.SimpleHTTPRequestHandler
# KML file name
KML_FILE = os.path.join(BASE_DIR, "soil_type.kml") # Change the parenthesized file name to your file name
KML_URL = f"http://localhost:{PORT}/{KML_FILE}"
KML_PATH = os.path.join(BASE_DIR, KML_FILE)

# Code an the HTML file that references the local KML file
def generate_html(output_html="index.html"):
    html_content = f'''<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Leaflet KML Map</title>
    <link rel="stylesheet" href="https://unpkg.com/leaflet/dist/leaflet.css" />
    <script src="https://unpkg.com/leaflet/dist/leaflet.js"></script>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/leaflet-omnivore/0.3.4/leaflet-omnivore.min.js"></script>
    <style>
        #map {{ height: 880px; }}
    </style>
</head>
<body>
    <div id="map"></div>
    <script>
        var map = L.map('map').setView([39.578, -95.261], 5);
        L.tileLayer('https://tile.openstreetmap.org/{{z}}/{{x}}/{{y}}.png', {{
            attribution: '&copy; OpenStreetMap contributors'
        }}).addTo(map);
        
        // Load KML from the local server
        var kmlLayer = omnivore.kml('http://localhost:{"KML_URL"}').on('ready', function() {{
            map.fitBounds(kmlLayer.getBounds());
        }}).addTo(map);
    </script>
</body>
</html>'''
    
    output_path = os.path.join(BASE_DIR, output_html)
    with open(output_path, "w", encoding="utf-8") as f:
        f.write(html_content)
    
    print(f"HTML file generated: {output_path}")
    return output_path

# Start a simple HTTP server
def start_server():
    os.chdir(BASE_DIR)  # Serve files from the script's directory
    handler = http.server.SimpleHTTPRequestHandler
    with socketserver.TCPServer(("", PORT), handler) as httpd:
        print(f"Serving at http://localhost:{PORT}")
        httpd.serve_forever()

# Run the server in a separate thread
def run():
    # Generate the HTML file
    html_path = generate_html()
    
    # Start the server in a separate thread
    server_thread = threading.Thread(target=start_server, daemon=True)
    server_thread.start()
    
    # Open the HTML file in the browser
    webbrowser.open(f"http://localhost:{PORT}/index.html")

if __name__ == "__main__":
    run()

HTML file generated: C:\Users\zachh\Desktop\cas502_project\index.html
Serving at http://localhost:8000


127.0.0.1 - - [02/Mar/2025 17:15:33] "GET /index.html HTTP/1.1" 200 -
